In [100]:
import glob
import pandas as pd
from datetime import datetime

In [101]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

In [102]:
cols=['Name','Market Cap (US$ Billion)']

In [103]:
def extract():
    df = pd.DataFrame(cols)
    df = extract_from_json("bank_market_cap_1.json")
    return df

In [104]:
# Write your code here
def extract_exchange_rates():
    exchange_rate = pd.read_csv("exchange_rates.csv")
    exchange_rate = exchange_rate.rename(columns={"Unnamed: 0":"Currency"})
    return exchange_rate

In [105]:
def transform(market_cap):
    rates = extract_exchange_rates()
    gbp_rate = float(rates.iloc[9,1])
    market_cap["Market Cap (US$ Billion)"] = round(market_cap["Market Cap (US$ Billion)"].multiply(gbp_rate), 3)
    market_cap = market_cap.rename(columns={"Market Cap (US$ Billion)":"Market Cap (GBP$ Billion)"})
    return market_cap

In [106]:
def load(bank_market_cap):
    bank_market_cap.to_csv("bank_market_cap_gbp.csv", index=False)
    

In [107]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

In [108]:
# Write your code here
log("ETL Job Started")
log("Extract phase Started")

In [109]:
# Call the function here
extracted_data = extract()

# Print the rows here
extracted_data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


In [110]:
# Write your code here
log("Extract phase Ended")

In [111]:
# Write your code here
log("Transform phase Started")

In [112]:
# Call the function here
transformed_data = transform(extracted_data)
# Print the first 5 rows here
transformed_data.head()

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


In [113]:
# Write your code here
log("Transform phase Ended")

In [114]:
# Write your code here
log("Load phase Started")

In [117]:
# Write your code here
load(transformed_data)

In [116]:
# Write your code here
log("Load phase Ended")